In [12]:
import pandas as pd
import os
import talib
import yfinance as yf

Total Files in Folder

In [13]:
list_dir = os.listdir("../Dataset/candlestic data")
len(list_dir)

4573

Remove Empty File

In [7]:
directory = "../Dataset/candlestic data"

# Get the list of files in the directory
file_list = os.listdir(directory)

# Iterate over the files
for filename in file_list:
    # Check if the file is a CSV file
    if filename.endswith(".csv"):
        file_path = os.path.join(directory, filename)
        # Read the CSV file with pandas
        df = pd.read_csv(file_path)
        # Check if the dataframe is empty (doesn't have any data rows)
        if df.empty:
            # Remove the empty CSV file
            os.remove(file_path)
            print(f"Removed empty file: {filename}")


Total Files after removing empty files

In [26]:
list_dir = os.listdir("../Dataset/candlestic data")
len(list_dir)

4573

Combine CSV file and make one csv

In [18]:
combined_data = pd.DataFrame()
total_companies = 100
for file in list_dir[:total_companies]:
    df = pd.read_csv("../Dataset/candlestic data/"+file)
    # df['Symbol'] = file.split(" ")[0]

    # Fetch fundamental data for a stock symbol
    fundamental_data = yf.Ticker(file.split(" ")[0]).info

    df['marketCap'] = fundamental_data.get("marketCap", None)
    df['PERatio'] = fundamental_data.get("trailingPE", None)

    # calculate and add technical indicators
    df['ATR'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    df['AD'] = talib.AD(df['High'], df['Low'], df['Close'], df['Volume'])
    df['AROON_DOWN'], df['AROON_UP'] = talib.AROON(df['High'], df['Low'], timeperiod=14)
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['STOCH_K'], df['STOCH_D'] = talib.STOCH(df['High'], df['Low'], df['Close'])
    macd, macd_signal, _ = talib.MACD(df['Close'])
    df['MACD'] = macd
    df['MACD_SIGNAL'] = macd_signal
    df['HAMMER'] = talib.CDLHAMMER(df['Open'], df['High'], df['Low'], df['Close'])
    df['INVERTED_HAMMER'] = talib.CDLINVERTEDHAMMER(df['Open'], df['High'], df['Low'], df['Close'])
    df['ENGULFING'] = talib.CDLENGULFING(df['Open'], df['High'], df['Low'], df['Close'])

    df.insert(0, "Symbol", file.split(" ")[0])
    combined_data = pd.concat([combined_data,df], ignore_index=True)
combined_data

,Symbol,Timestamp,Open,High,Low,Close,Volume,marketCap,PERatio,ATR,...,AROON_DOWN,AROON_UP,ADX,STOCH_K,STOCH_D,MACD,MACD_SIGNAL,HAMMER,INVERTED_HAMMER,ENGULFING
0,AACG,2011-01-03 00:00:00-05:00,0.294686,0.301913,0.294686,0.297095,2400,48958144,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,AACG,2011-01-04 00:00:00-05:00,0.293883,0.300307,0.293080,0.300307,1800,48958144,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
2,AACG,2011-01-05 00:00:00-05:00,0.303519,0.306730,0.293883,0.297095,6400,48958144,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,AACG,2011-01-06 00:00:00-05:00,0.301110,0.313154,0.300307,0.308336,11500,48958144,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,AACG,2011-01-07 00:00:00-05:00,0.305125,0.309139,0.305125,0.309139,11500,48958144,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170301,AEP,2022-12-23 00:00:00-05:00,92.543243,93.996445,92.376323,93.957169,1406200,44272025600,22.337664,2.048413,...,64.285714,42.857143,28.784589,69.318965,44.504714,0.782064,1.249709,0,0,100
170302,AEP,2022-12-27 00:00:00-05:00,94.232098,94.713222,93.515314,94.546303,1327300,44272025600,22.337664,1.987663,...,57.142857,35.714286,28.022943,86.259184,65.294818,0.794480,1.158663,0,0,0
170303,AEP,2022-12-28 00:00:00-05:00,94.742683,95.105985,93.574237,93.633148,1391800,44272025600,22.337664,1.955097,...,50.000000,28.571429,27.497742,85.039766,80.205972,0.722309,1.071392,0,0,-100
170304,AEP,2022-12-29 00:00:00-05:00,94.192824,94.909608,93.937531,94.271378,1415900,44272025600,22.337664,1.906623,...,42.857143,21.428571,27.010055,78.239241,83.179397,0.708446,0.998803,0,0,0


In [33]:
combined_data.dropna(inplace=True)
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1502403 entries, 33 to 1535502
Data columns (total 20 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Symbol           1502403 non-null  object 
 1   Timestamp        1502403 non-null  object 
 2   Open             1502403 non-null  float64
 3   High             1502403 non-null  float64
 4   Low              1502403 non-null  float64
 5   Close            1502403 non-null  float64
 6   Volume           1502403 non-null  int64  
 7   ATR              1502403 non-null  float64
 8   RSI              1502403 non-null  float64
 9   AD               1502403 non-null  float64
 10  AROON_DOWN       1502403 non-null  float64
 11  AROON_UP         1502403 non-null  float64
 12  ADX              1502403 non-null  float64
 13  STOCH_K          1502403 non-null  float64
 14  STOCH_D          1502403 non-null  float64
 15  MACD             1502403 non-null  float64
 16  MACD_SIGNAL      

In [19]:
combined_data.to_csv(f"../Dataset/process data/{total_companies} companies.csv", index=False)